In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chicken Disease Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Projects\\Chicken Disease Classification'

In [7]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir : Path 
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

    

In [8]:
from CDC.constants import *
from CDC.utils.helper_functions import read_yaml, create_directories
from CDC.entity.config_entity import DataIngestionConfig, BaseModelConfig, CallbacksConfig
import os
from box import ConfigBox
import tensorflow as tf 


class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> ConfigBox:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            unzip_dir = config.unzip_dir,
            local_data_file = config.local_data_file,
            source_url = config.source_URL
        )

        return data_ingestion_config
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model
        create_directories([config.root_dir])
        base_model_config = BaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.INPUT_SHAPE,
            params_batch_size = self.params.BATCH_SIZE,
            params_epochs = self.params.EPOCHS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_classes = self.params.CLASSES,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS
        )
        
        return base_model_config

    def get_callbacks_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([config.tensorboard_root_log_dir,
                            model_checkpoint_dir])
        callbacks_config = CallbacksConfig(
            root_dir = Path(config.root_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir)
        )
        return callbacks_config

    
    def get_training_config(self):
        training = self.config.training
        base_model = self.config.base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(training.root_dir)])
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            training_data = Path(training_data),
            updated_base_model_path = Path(base_model.updated_base_model_path),
            params_epochs = params.EPOCHS,
            params_image_size = params.INPUT_SHAPE,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION
        )

        return training_config



In [9]:
import time



In [15]:
import time
import tensorflow as tf
from CDC import logger
from pathlib import Path
from CDC.entity.config_entity import CallbacksConfig
import os


class CallBacks:
    def __init__(self,config: CallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
                    self._create_tb_callbacks,
                    self._create_ckpt_callbacks
        ]








In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [19]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    callbacks = CallBacks(config=callbacks_config)
    callback_list = callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-06-21 19:21:38,258: INFO: helper_functions: yaml file config\config.yaml uploaded successfully]
[2023-06-21 19:21:38,265: INFO: helper_functions: yaml file params.yaml uploaded successfully]
[2023-06-21 19:21:38,268: INFO: helper_functions: created directory at: artifacts]
[2023-06-21 19:21:38,271: INFO: helper_functions: created directory at: artifacts/callbacks/tensorboard_log_dir]
[2023-06-21 19:21:38,274: INFO: helper_functions: created directory at: artifacts/callbacks/checkpoint_dir]
[2023-06-21 19:21:38,277: INFO: helper_functions: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/3
19/19 [==============================] - 160s 9s/step - loss: 13.4940 - accuracy: 0.5135 - val_loss: 4.1889 - val_accuracy: 0.4688
Epoch 2/3
19/19 [==============================] - 185s 10s/step - loss: 8.7346 - accuracy: 0.5574 - val_loss: 2.6457 - val_accuracy: 0.7656
Epoch 3/3
19/19 [===========================